# Решение задачи NER (Named-entity recognition): удаление персональных данных с использованием обученной модели

### Импортируем необходимые библиотеки

In [1]:
from transformers import pipeline
import string

2024-07-16 12:38:08.478389: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-16 12:38:08.527918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 12:38:09.284244: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Загружаем модель 

In [2]:
pipe = pipeline("token-classification", model="mdarhri00/named-entity-recognition") 

/home/osipov/.local/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Создаем текст, из которого хотим удалить персональные данные. В качестве примера возмем несуществующего человека с несуществующими данными

In [3]:
text = """Уважаемый Иванов Иван, Я Сергеев Сергей Сергеевич,пенсионер, проживаю Московская обл. с.Крутое, ул.Пушкина дом Колотушкина обращаюсь к Вам с прозьбой 
разобраться с нашим интернетом.Я заплатил за это 1 рубль. Три месяца назад обратился в районный сервис с проблемой низкой скоростью. Поставили на очередь в областной центр к специалисту и вот
 в течении трех месяцев жду вызов.За время ожидания парсеры совсем перестали работать. Пишу за себя,но знаю что идет обман многих. Прозьба понять и сделать вывод. С большим Уважением .Сергей.т +79909090900. я знаю как они звонят, скажут просто что трубку не берут,мой  № 89909090900 ,мой  №89909090900. Мой емайл адрес - ivanov@example.com. Жду"""

In [4]:
result = pipe(text)

### Выделяем сущности, которые хотим удалить

In [5]:
censorship_entity = ['Person_Name', 'Identification_Number', 'phone_number', 'email', 'Brand_vehicule', 'location']

In [6]:
special_characters = string.punctuation

### Создаем функцию замены персональных данных на звездочки

In [7]:
def replace_with_asterisks(text, start, end):
  return text[:start] + "*" * (end - start) + text[end:]

### Заменяем

In [8]:
for data in result:
    if data['entity'] in censorship_entity and data['word'] not in special_characters:
        text = replace_with_asterisks(text, data['start'], data['end'])

### Итог

In [9]:
text

'Уважаемый ****** ****, Я ******* ****** *********,пенсионер, проживаю ********** ***. *.******, **.******* дом *********** обращаюсь к Вам с прозьбой \nразобраться с нашим интернетом.Я заплатил за это 1 рубль. Три месяца назад обратился в районный сервис с проблемой низкой скоростью. Поставили на очередь в областной центр к специалисту и вот\n в течении трех месяцев жду вызов.За время ожидания парсеры совсем перестали работать. Пишу за себя,но знаю что идет обман многих. Прозьба понять и сделать вывод. С большим Уважением .******.т +***********. я знаю как они звонят, скажут просто что трубку не берут,мой  № *********** ,мой  №***********. \n Мой емайл адрес - ******@*******.***. Жду'

### Конечно, можно было бы применить дополнительно какую-либо модель генерации текста с возможным промтом "Удали звездочки и приведи текст в нормальную форму" для формирования качественного текста и последующего использования. Однако здесь я лишь хотел показать один из способов удаления персональных данных. 